In [11]:
import pandas as pd
import os

def backtest_metrics(file_paths, total_capital=30_000_000, risk_free_rate=0.035):
    all_trades = []
    for path in file_paths:
        stock = os.path.basename(path).split('.')[0]
        df = pd.read_csv(path)
        df['stock'] = stock
        df['entry'] = pd.to_datetime(df['entry'])
        df['exit'] = pd.to_datetime(df['exit'])
        all_trades.append(df)

    df_all = pd.concat(all_trades, ignore_index=True)
    df_all['holding_days'] = (df_all['exit'] - df_all['entry']).dt.days
    df_all.sort_values('exit', inplace=True)

    capital_per_stock = total_capital / len(file_paths)

    nav_list = []
    for _, row in df_all.iterrows():
        nav_list.append({'date': row['exit'],
            'stock': row['stock'],
            'pnl': row['net_pnl'] / capital_per_stock })

    nav_df = pd.DataFrame(nav_list)
    nav_df = nav_df.pivot(index='date', columns='stock', values='pnl').fillna(0)
    nav_df = nav_df.cumsum()
    portfolio_nav = (1 + nav_df.mean(axis=1))  

    daily_returns = portfolio_nav.pct_change().dropna()

    avg_holding = df_all['holding_days'].mean()
    ann_return = portfolio_nav.iloc[-1] ** (252 / len(portfolio_nav)) - 1
    ann_volatility = daily_returns.std() * (252 ** 0.5)
    sharpe = (ann_return - risk_free_rate) / ann_volatility

    return {"Average Holding Period (days)": avg_holding,
        "Annual Return": ann_return,
        "Annual Volatility": ann_volatility,
        "Sharpe Ratio": sharpe}


if __name__ == "__main__":
    folder = os.path.expanduser("~/Downloads/stock_data")
    paths = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".csv")]

    results = backtest_metrics(paths)
    print("\nresult")
    for k, v in results.items():
        print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


result
Average Holding Period (days): 1.4555
Annual Return: 0.1600
Annual Volatility: 0.0635
Sharpe Ratio: 1.9696
